In [12]:
%config IPCompleter.greedy=True

In [1]:
import cvutilities.openpose_utilities
import cvutilities.camera_utilities
import numpy as np
import pickle
import pandas as pd
import time
import os

In [2]:
# Define input and output files
data_directory = './data'
input_dataframe_filename = 'example_2d_poses_dataframe.pickle.xz'
output_dataframe_filename = 'example_3d_pose_tracks_dataframe.pickle.xz'

input_dataframe_path = os.path.join(
    data_directory,
    input_dataframe_filename)

output_dataframe_path = os.path.join(
    data_directory,
    output_dataframe_filename)

In [3]:
# Define model parameters

room_size = np.array([7.0, 8.0, 3.0])

pose_initialization_model = cvutilities.openpose_utilities.PoseInitializationModel(
    initial_keypoint_position_means = np.tile(room_size/2, (18, 1)),
    initial_keypoint_velocity_means = np.zeros((18,3)),
    initial_keypoint_position_error = np.amax(room_size)/2.0,
    initial_keypoint_velocity_error = 2.0)

keypoint_model = cvutilities.openpose_utilities.KeypointModel(
    position_observation_error = 1.0,
    reference_delta_t = 0.1,
    reference_position_transition_error = 0.1,
    reference_velocity_transition_error = 0.1)

pose_tracking_model = cvutilities.openpose_utilities.PoseTrackingModel(
    cost_threshold = 1.0,
    num_missed_observations_threshold = 10)

In [4]:
# Ingest and process the data

input_dataframe = pd.read_pickle(input_dataframe_path)

num_rows = input_dataframe.shape[0]

camera_names = input_dataframe.columns.levels[0].tolist()

camera_calibration_parameters = cvutilities.camera_utilities.fetch_camera_calibration_data_from_local_drive_multiple_cameras(
    camera_names = camera_names,
    camera_calibration_data_directory = './data')

initial_dataframe_row = input_dataframe.iloc[0]

initial_poses_2d = cvutilities.openpose_utilities.Poses2D.from_dataframe_row(
    dataframe_row = initial_dataframe_row,
    camera_names = camera_names)

initial_poses_3d = cvutilities.openpose_utilities.Poses3D.from_poses_2d(
    poses_2d = initial_poses_2d,
    cameras = camera_calibration_parameters)

pose_tracks = cvutilities.openpose_utilities.Pose3DTracks.initialize(
    pose_initialization_model = pose_initialization_model,
    keypoint_model = keypoint_model,
    pose_tracking_model = pose_tracking_model,
    pose_3d_observations = initial_poses_3d)

start_time = time.time()
for row_index in range(1, num_rows):
    dataframe_row = input_dataframe.iloc[row_index]
    poses_2d = cvutilities.openpose_utilities.Poses2D.from_dataframe_row(
        dataframe_row,
        camera_names)
    poses_3d = cvutilities.openpose_utilities.Poses3D.from_poses_2d(
        poses_2d,
        camera_calibration_parameters)
    pose_tracks.update(poses_3d)
end_time = time.time()
elapsed_time = end_time - start_time

print('{} tracks produced from {} frames in {:.1f} seconds: {:.1f} milliseconds per frame'.format(
    pose_tracks.num_inactive_tracks() + pose_tracks.num_active_tracks(),
    num_rows,
    elapsed_time,
    1000*elapsed_time/num_rows))

output_dataframe = pose_tracks.dataframe()

output_dataframe.to_pickle(output_dataframe_path)

print('Output saved in {}'.format(output_dataframe_filename))

57 tracks produced from 600 frames in 17.9 seconds: 29.9 milliseconds per frame
Output saved in example_3d_pose_tracks_dataframe.pickle.xz


In [9]:
input_dataframe.columns

MultiIndex([('camera01', 'poses'),
            ('camera01',  'file'),
            ('camera01', 'model'),
            ('camera01', 'frame'),
            ('camera02', 'poses'),
            ('camera02',  'file'),
            ('camera02', 'model'),
            ('camera02', 'frame'),
            ('camera03', 'poses'),
            ('camera03',  'file'),
            ('camera03', 'model'),
            ('camera03', 'frame'),
            ('camera04', 'poses'),
            ('camera04',  'file'),
            ('camera04', 'model'),
            ('camera04', 'frame')],
           )

In [15]:
input_dataframe.dtypes

camera01  poses      object
          file     category
          model    category
          frame       int64
camera02  poses      object
          file     category
          model    category
          frame       int64
camera03  poses      object
          file     category
          model    category
          frame       int64
camera04  poses      object
          file     category
          model    category
          frame       int64
dtype: object

In [17]:
cam1 = input_dataframe['camera01']

In [27]:
len(cam1['file'])

600

In [39]:
len(cam1['file'].values.unique())

6

In [44]:
cam1

,poses,file,model,frame
2018-07-04 18:20:00.000,"[[[1098.0, 216.58696, 0.81464326], [1164.0, 24...",video_2018-07-04-18-20-00.mp4,cmu,0
2018-07-04 18:20:00.100,"[[[1092.0, 221.86957, 0.92140007], [1158.0, 25...",video_2018-07-04-18-20-00.mp4,cmu,1
2018-07-04 18:20:00.200,"[[[1086.0, 227.15218, 0.8493447], [1152.0, 253...",video_2018-07-04-18-20-00.mp4,cmu,2
2018-07-04 18:20:00.300,"[[[1074.0, 227.15218, 0.74878925], [1140.0, 25...",video_2018-07-04-18-20-00.mp4,cmu,3
2018-07-04 18:20:00.400,"[[[924.0, 47.54348, 0.9911146], [954.0, 100.36...",video_2018-07-04-18-20-00.mp4,cmu,4
...,...,...,...,...
2018-07-04 18:20:59.500,"[[[774.0, 31.695652, 0.93963164], [768.0, 68.6...",video_2018-07-04-18-20-50.mp4,cmu,95
2018-07-04 18:20:59.600,"[[[774.0, 31.695652, 0.9273625], [762.0, 68.67...",video_2018-07-04-18-20-50.mp4,cmu,96
2018-07-04 18:20:59.700,"[[[774.0, 36.97826, 0.8920598], [762.0, 73.956...",video_2018-07-04-18-20-50.mp4,cmu,97
2018-07-04 18:20:59.800,"[[[0.0, 0.0, 0.0], [1200.0, 369.78262, 0.82549...",video_2018-07-04-18-20-50.mp4,cmu,98


# Video input:
- 4 cameras
- 1 minute of video is recorded for each camera
- 1 minute of video footage is broken into 6 videos
- Each video records 10 seconds
- 10 seconds of video has 100 frames
- 10 frames per second

In [42]:
output_dataframe.columns

Index(['track_id', 'track_status', 'timestamp', 'keypoints', 'valid_keypoints',
       'keypoint_std_devs', 'tag'],
      dtype='object')

In [43]:
output_dataframe

,track_id,track_status,timestamp,keypoints,valid_keypoints,keypoint_std_devs,tag
0,4,inactive,2018-07-04T18:20:00.600000,"[[3.200903724221622, 4.03407871021944, 1.68237...","[True, True, True, True, True, True, True, Tru...","[[0.970142500145332, 0.970142500145332, 0.9701...",None
1,4,inactive,2018-07-04T18:20:00.700000,"[[3.200903724221622, 4.03407871021944, 1.68237...","[True, True, True, True, True, True, True, Tru...","[[0.9955784602874027, 0.9955784602874027, 0.99...",None
2,4,inactive,2018-07-04T18:20:00.800000,"[[3.200903724221622, 4.03407871021944, 1.68237...","[True, True, True, True, True, True, True, Tru...","[[1.0589034283579573, 1.0589034283579573, 1.05...",None
3,4,inactive,2018-07-04T18:20:00.900000,"[[3.200903724221622, 4.03407871021944, 1.68237...","[True, True, True, True, True, True, True, Tru...","[[1.1539828727447543, 1.1539828727447543, 1.15...",None
4,4,inactive,2018-07-04T18:20:01.000000,"[[3.200903724221622, 4.03407871021944, 1.68237...","[True, True, True, True, True, True, True, Tru...","[[1.2738039372635948, 1.2738039372635948, 1.27...",None
...,...,...,...,...,...,...,...
2159,56,active,2018-07-04T18:20:59.500000,"[[3.5, 4.0, 1.5], [3.5, 4.0, 1.5], [3.5, 4.0, ...","[True, True, True, True, True, True, True, Tru...","[[4.0, 4.0, 4.0], [4.0, 4.0, 4.0], [4.0, 4.0, ...",None
2160,56,active,2018-07-04T18:20:59.600000,"[[5.013853698182316, 6.1111414425533885, 1.468...","[True, True, True, True, True, True, True, Tru...","[[0.9702314020928409, 0.9702314020928409, 0.97...",None
2161,56,active,2018-07-04T18:20:59.700000,"[[5.062811014914076, 6.179414764072325, 1.4672...","[True, True, True, True, True, True, True, Tru...","[[0.706406290024958, 0.706406290024958, 0.7064...",None
2162,56,active,2018-07-04T18:20:59.800000,"[[5.083235562294404, 6.2078977730128715, 1.466...","[True, True, True, True, True, True, True, Tru...","[[0.609366514245326, 0.609366514245326, 0.6093...",None


In [47]:
len(output_dataframe['track_id'].unique())

57

In [48]:
output_dataframe['tag'].unique()

array([None], dtype=object)

In [49]:
output_dataframe['track_status'].unique()

array(['inactive', 'active'], dtype=object)

In [51]:
output_dataframe['timestamp']

0       2018-07-04T18:20:00.600000
1       2018-07-04T18:20:00.700000
2       2018-07-04T18:20:00.800000
3       2018-07-04T18:20:00.900000
4       2018-07-04T18:20:01.000000
                   ...            
2159    2018-07-04T18:20:59.500000
2160    2018-07-04T18:20:59.600000
2161    2018-07-04T18:20:59.700000
2162    2018-07-04T18:20:59.800000
2163    2018-07-04T18:20:59.900000
Name: timestamp, Length: 2164, dtype: object

In [57]:
3*600

1800

In [54]:
len(output_dataframe[output_dataframe['track'].unique() == 'active'])

163

In [56]:
output_dataframe['track_id'].unique()

array([ 4,  3,  5,  6,  7,  8,  9, 10,  1,  2, 14,  0, 12, 16, 17, 18, 19,
       20, 21, 13, 23, 24, 22, 15, 25, 26, 28, 31, 11, 30, 27, 34, 29, 33,
       37, 32, 35, 40, 41, 42, 36, 43, 39, 45, 44, 47, 38, 46, 48, 50, 49,
       53, 52, 51, 54, 55, 56])